In [1]:
import pandas as pd
import matplotlib.colors as pltcolors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec
from datetime import datetime
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import copy
from datetime import datetime
import random

#Lectura de identificadores
#Como los archivos descargados siguen un formato que involucra el identificador, es util tener a mano estos datos para agrupar los datos

f = open("ID_Estaciones.txt", "r")
ids = []
lines = f.readlines()
ids_dict = {}
estaciones = {}

for line in lines:
    temp = line.split(",")
    ids.append(line.split(",")[len(temp) - 1].split("\n")[0])
    ids_dict[line.split(",")[len(temp) - 1].split("\n")[0]] = {}
    estaciones[line.split(",")[len(temp) - 1].split("\n")[0]] = temp

print(ids)
print(ids_dict)
labels = copy.deepcopy(ids)

['510005', '400009', '180005', '330020', '550001', '270008', '290004', '410005', '430002', '430009', '450001', '450005', '530005', '200006', '330113']
{'510005': {}, '400009': {}, '180005': {}, '330020': {}, '550001': {}, '270008': {}, '290004': {}, '410005': {}, '430002': {}, '430009': {}, '450001': {}, '450005': {}, '530005': {}, '200006': {}, '330113': {}}


In [2]:
bigDf = pd.DataFrame()
for id in ids:
    df = pd.read_excel(id+"_agrupado.xlsx", sheet_name="Sheet1")
    if(bigDf.size == 0):
        bigDf = pd.concat([df.iloc[:,1:]], ignore_index = True)
    else:
        bigDf = pd.concat([bigDf,df.iloc[:,1:]], ignore_index = True)

bigDf["momento"] = pd.to_datetime(bigDf["momento"], format="%d-%m-%Y %H:%M:%S")
bigDf["Nombre_Estacion"] = ""
bigDf["Region"] = ""
for key,value in estaciones.items():
    bigDf.loc[bigDf["ID_Estacion"] == int(key), "Nombre_Estacion"] = value[0]
    bigDf.loc[bigDf["ID_Estacion"] == int(key), "Region"] = value[1]

bigDf

solar = pd.DataFrame()
df = pd.read_excel("Paneles_Solartex.xlsx", sheet_name="Hoja1")

if(solar.size == 0):
    solar = pd.concat([df.iloc[:,0:]], ignore_index = True)
else:
    solar = pd.concat([solar,df.iloc[:,0:]], ignore_index = True)

ef = (solar.iloc[:,1] / (1000*((solar.iloc[:,3]/1000) * (solar.iloc[:,2]/1000))))*100

solar.insert(2,"Eficiencia(%)",ef,True)

aero = pd.DataFrame()
df = pd.read_excel("Paneles_Solartex.xlsx", sheet_name="Hoja3")
if(aero.size == 0):
    aero = pd.concat([df.iloc[:,0:]], ignore_index = True)
else:
    aero = pd.concat([aero,df.iloc[:,0:]], ignore_index = True)


for x in range(solar.shape[0]):
    temp = solar.iloc[x,:]
    potencia = ((temp["Ancho(mm)"]/1000) * (temp["Alto(mm)"]/1000)) * bigDf["RadGInst (watt/m2)"] * (temp["Eficiencia(%)"]/100)
    bigDf.insert(bigDf.shape[1],"Panel Solar "+str(x),potencia,True)

def translateWind(value, fromMin, fromMax, toMin, toMax, clause):
    tmp = value * 0.514444
    # Figure out how 'wide' each range is
    tmp = pd.DataFrame(tmp)
    tmp["potencia"] = 0
    #display(tmp)
    fromSpan = fromMax - fromMin
    toSpan = toMax - toMin

    # Convert the left range into a 0-1 range (float)
    tmp["potencia"] = (tmp["ff_Valor(kt)"] - fromMin) / (fromSpan)
    tmp["potencia"] = toMin + (tmp["potencia"]  * toSpan)

    tmp.loc[tmp["ff_Valor(kt)"] < fromMin, "potencia"] = 0
    tmp.loc[tmp["ff_Valor(kt)"] >= fromMax, "potencia"] = toMax
    tmp.loc[tmp["ff_Valor(kt)"] >= clause, "potencia"] = 0

    # Convert the 0-1 range into a value in the right range.
    return tmp

for x in range(aero.shape[0]):
    temp = aero.iloc[x,:]
    #display(temp)
    a = translateWind(bigDf["ff_Valor(kt)"],temp["Viento de arranque(m/s)"], temp["Viento nominal(m/s)"],0,temp["Potencia nominal(W)"],temp["Viento de corte(m/s)"])
    bigDf.insert(bigDf.shape[1],"Aerogenerador "+str(x),a["potencia"],True)
bigDf["ff_Valor(kt)"] = bigDf["ff_Valor(kt)"] * 0.514444
bigDf.rename(columns={"ff_Valor(kt)":"ff_Valor(m/s)"},inplace=True)


display(solar, aero, bigDf)


,Precio,Potencia Maxima(W),Eficiencia(%),Ancho(mm),Alto(mm),Fondo(mm),Nombre
0,14548,10,10.822511,280,330,18,Panel Solar 10 Watts Policristalino RESUN
1,19838,20,10.688100,652,287,25,Panel Solar 20 Watts Monocristalino Restarsolar
2,20630,25,13.477089,530,350,20,Panel Solar 25 Watts Policristalino YI SOLAR
3,29990,30,17.857143,480,350,25,Panel Solar 30 Watts Eco Delta Monocristalino
4,32400,50,13.819790,670,540,30,Panel Solar 50 Watts Policristalino RESUN
5,53750,100,14.632719,1020,670,35,Panel Solar 100 Watts Monocristalino RESUN
6,54000,100,14.632719,1020,670,30,Panel Solar 100 Watts Monocristalino RESUN
7,54000,50,13.819790,670,540,25,Panel Solar 50 Watts Monocristalino RESUN
8,53000,100,14.632719,1020,670,30,Panel Solar 100 Watts Policristalino RESUN
9,69000,150,15.146283,1487,666,35,Panel Solar 150 Watts Policristalino Resun


,Nombre,Precio,Diametro Rotor(mm),Viento de arranque(m/s),Viento nominal(m/s),Viento de corte(m/s),Potencia nominal(W)
0,Generador Eólico 3 Aspas 400 Watts 24V,310270,1200,2.5,12.0,40,400
1,Generador Eólico 400W 3 Aspas con Salida a 24V DC,409990,1200,2.0,12.0,40,400
2,Generador Eólico 400W 3 Aspas con Salida a 24V AC,429990,1300,2.0,13.0,50,400
3,Generador Eólico 600W Max 650W 48V 3 Aspas,647990,1700,2.5,12.0,40,600
4,Generador Eólico 1000W Max 48V 3 Aspas,1499990,2000,3.0,10.0,45,1000
5,Generador Eólico 600W Max 650W 24V 3 Aspas,647990,1800,2.5,12.0,40,600
6,Generador Eólico 2000W Max. 48V AC 3 Aspas,2699990,3200,2.5,10.0,40,2000
7,AEROGENERADOR - TURBINA EOLICA - ALTO RENDIMIE...,2737000,2000,2.0,7.0,50,600
8,AEROGENERADOR - TURBINA EOLICA - ALTO RENDIMIE...,3808000,2500,2.5,8.0,50,1000
9,AEROGENERADOR - TURBINA EOLICA - ALTO RENDIMIE...,5831000,2500,2.5,8.0,50,2000


,ID_Estacion,momento,RR6_Valor(mm),HR_Valor(%),Ts_Valor(°C),ff_Valor(m/s),RadGInst (watt/m2),Nombre_Estacion,Region,Panel Solar 0,...,Aerogenerador 4,Aerogenerador 5,Aerogenerador 6,Aerogenerador 7,Aerogenerador 8,Aerogenerador 9,Aerogenerador 10,Aerogenerador 11,Aerogenerador 12,Aerogenerador 13
0,510005,2023-01-01 00:00:00,0.0,57.8,9.7,11.832212,78.1,EstaciÃ³n Teniente Gallardo Puerto Natales Ad.,Region de Magallanes y de la Antartica Chilena,0.781,...,1000.000000,589.402863,2000.000000,600.00000,1000.000000,2000.000000,392.935242,392.935242,373.28848,392.935242
1,510005,2023-01-01 01:00:00,NaN,61.6,8.8,11.317768,7.7,EstaciÃ³n Teniente Gallardo Puerto Natales Ad.,Region de Magallanes y de la Antartica Chilena,0.077,...,1000.000000,556.911663,2000.000000,600.00000,1000.000000,2000.000000,371.274442,371.274442,352.71072,371.274442
2,510005,2023-01-01 02:00:00,NaN,62.0,8.1,10.288880,0.0,EstaciÃ³n Teniente Gallardo Puerto Natales Ad.,Region de Magallanes y de la Antartica Chilena,0.000,...,1000.000000,491.929263,2000.000000,600.00000,1000.000000,2000.000000,327.952842,327.952842,311.55520,327.952842
3,510005,2023-01-01 03:00:00,NaN,59.6,8.7,10.288880,0.0,EstaciÃ³n Teniente Gallardo Puerto Natales Ad.,Region de Magallanes y de la Antartica Chilena,0.000,...,1000.000000,491.929263,2000.000000,600.00000,1000.000000,2000.000000,327.952842,327.952842,311.55520,327.952842
4,510005,2023-01-01 04:00:00,NaN,58.8,8.9,8.745548,0.0,EstaciÃ³n Teniente Gallardo Puerto Natales Ad.,Region de Magallanes y de la Antartica Chilena,0.000,...,820.792571,394.455663,1665.479467,600.00000,1000.000000,2000.000000,262.970442,262.970442,249.82192,262.970442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129908,330113,2023-12-31 19:00:00,NaN,38.6,27.6,2.572220,898.1,EstaciÃ³n El Paico,Region Metropolitana,8.981,...,0.000000,4.561263,19.258667,68.66640,13.130909,26.261818,3.040842,3.040842,2.88880,3.040842
129909,330113,2023-12-31 20:00:00,NaN,38.6,27.4,3.601108,712.7,EstaciÃ³n El Paico,Region Metropolitana,7.127,...,85.872571,69.543663,293.628800,192.13296,200.201455,400.402909,46.362442,46.362442,44.04432,46.362442
129910,330113,2023-12-31 21:00:00,NaN,41.3,25.6,4.629996,492.1,EstaciÃ³n El Paico,Region Metropolitana,4.921,...,232.856571,134.526063,567.998933,315.59952,387.272000,774.544000,89.684042,89.684042,85.19984,89.684042
129911,330113,2023-12-31 22:00:00,NaN,48.4,23.7,3.086664,263.6,EstaciÃ³n El Paico,Region Metropolitana,2.636,...,12.380571,37.052463,156.443733,130.39968,106.666182,213.332364,24.701642,24.701642,23.46656,24.701642


In [3]:
def plot_avgViento():
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    meanDf = pd.DataFrame(columns=ids)
    err = True
    selected=ids
    
    for x in range(1,13):
        filtered_df = bigDf[bigDf['momento'].dt.month == x]
        temp = pd.DataFrame(columns=ids)
        for id in ids:
            est = filtered_df.query('ID_Estacion == '+id).dropna()
            #RadGInst (watt/m2)
            temp[id] = [est["ff_Valor(m/s)"].mean()]
            temp[id+"_std"] = [est["ff_Valor(m/s)"].std()]
        if(meanDf.size == 0):
            meanDf = pd.concat([temp], ignore_index=True)
        else:
            meanDf = pd.concat([meanDf,temp], ignore_index=True)

    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    errChk = widgets.Checkbox(description="Desviacion estandar?", value=err, indent=False, layout={'width':'90%'})
    plotOutput = widgets.Output()

    def errChange(change):
        nonlocal err, selected
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            err = not err
            replot(selected, err)
    
    def checkboxes(_):
        nonlocal err, selected
        selected = [label for label,chk in zip(ids,chkbxs) if chk.value]
        replot(selected, err)
            
    def replot(selected, dv):
        with plotOutput:
            plotOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            for id in selected:
                if(dv):
                    plt.errorbar(meses, meanDf[id], meanDf[id+"_std"], color=colours[id], capsize=5, fmt="-o", elinewidth=1, label = id)
                else:
                    plt.plot(meses, meanDf[id], label = id, color=colours[id], marker='o') 
            plt.xlabel("2023")
            plt.ylabel("Velocidad del viento(m/s)")
            plt.grid()
            #plt.ylim(0,100)
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.suptitle("Velocidad del viento promedio mensual\nAño 2023")
            plt.show()

    replot(selected,err)

    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    errChk.observe(errChange)
        
    V_in = widgets.VBox(chkbxs)
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'90%'})
    V_in2 =  widgets.VBox([ac_chk,errChk])
    H = widgets.HBox([plotOutput,V_in2], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea con desviacion estandar de la velocidad del viento promedio mensual''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    return V


def boxPlot_viento():

    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    mesIndex = 0
    estacion = ids[0]
    scale = True
    
    scaleChk = widgets.Checkbox(description="Escala?", value=scale)
    prevB = widgets.Button(description="Anterior")
    nextB = widgets.Button(description="Siguiente")
    mesDrop = widgets.Dropdown(
        options=meses,
        value='Ene',
        description='Mes:',
        disabled=False,
    )
    estacionDrop = widgets.Dropdown(
        options=ids,
        value=ids[0],
        description='Estacion:',
        disabled=False,
    )
    
    output = widgets.Output()
    def changed(change):
        nonlocal scale
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            scale = not scale
            replot(mesIndex,estacion, scale)
    
    def prev_click(b):
        nonlocal mesIndex, scale
        if(mesIndex == 11):
            nextB.disabled = False
        mesIndex = mesIndex - 1
        if(mesIndex <= 0):
            mesIndex = 0
            prevB.disabled = True
        mesDrop.value = meses[mesIndex]
        replot(mesIndex, estacion, scale)

    def next_click(b):
        nonlocal mesIndex, scale
        if(mesIndex == 0):
            prevB.disabled = False
        mesIndex = mesIndex + 1
        if(mesIndex >= 11):
            mesIndex = 11
            nextB.disabled = True
        mesDrop.value = meses[mesIndex]
        replot(mesIndex, estacion, scale)
    
    def meses_dropdown(change):
        nonlocal mesIndex, scale
        if change['type'] == 'change' and change['name'] == 'value':
            mesIndex = meses.index(change['new'])
            if(meses.index(change['new']) == 0):
                prevB.disabled = True
                nextB.disabled = False
            elif(meses.index(change['new']) == 11):
                prevB.disabled = False
                nextB.disabled = True
            else:
                prevB.disabled = False
                nextB.disabled = False
            replot(mesIndex,estacion, scale)
            print("changed to"+ change['new'] +" which is on index"+str(meses.index(change['new'])))

    def estacion_dropdown(change):
        nonlocal estacion, scale
        if(change['type'] == 'change' and change['name'] == 'value'):
            estacion = change['new']
            replot(mesIndex,estacion, scale)
            print("changed to "+ change['new'] +" which is on index "+str(ids.index(change['new'])))
    
    def replot(mes, estacion, scale):
        with output:
            output.clear_output(wait=True)
            fig = plt.figure(figsize=(8,4))
            gs = gridspec.GridSpec(3, 4, wspace=0.50)
            axs = []
            axs.append(fig.add_subplot(gs[:,0]))
            axs.append(fig.add_subplot(gs[:,1:3]))
            axs.append(fig.add_subplot(gs[:,3]))
            estDf = bigDf.query('ID_Estacion == '+estacion)
            
            est = estDf[estDf['momento'].dt.month == mes+1]
            rawData = axs[1].boxplot(est["ff_Valor(m/s)"].dropna())

            #plt.show()
            if(mes==0):
                est3 = estDf[estDf['momento'].dt.month == mes+2]
                axs[0].get_yaxis().set_visible(False)
                axs[2].boxplot(est3["ff_Valor(m/s)"].dropna())
                axs[2].set_title(meses[mes+1])
            elif(mes==11):
                est2 = estDf[estDf['momento'].dt.month == mes]
                axs[0].boxplot(est2["ff_Valor(m/s)"].dropna())
                axs[0].set_title(meses[mes-1])
                axs[2].get_yaxis().set_visible(False)
            else:
                est2 = estDf[estDf['momento'].dt.month == mes]
                est3 = estDf[estDf['momento'].dt.month == mes+2]
                axs[0].boxplot(est2["ff_Valor(m/s)"].dropna())
                axs[0].set_title(meses[mes-1])
                axs[2].boxplot(est3["ff_Valor(m/s)"].dropna())
                axs[2].set_title(meses[mes+1])
            for x in range(len(axs)):
                if(scale):
                    axs[x].set_ylim(0,40)
                axs[x].set_ylabel("Velocidad del viento(m/s)")
                axs[x].get_xaxis().set_visible(False)
                axs[x].grid()
            axs[1].set_title("Comportamiento del viento - Año 2023\nEstacion "+estacion+" - Periodo "+meses[mes])
            plt.show()

    replot(mesIndex,estacion,scale)
    nextB.on_click(next_click)
    prevB.on_click(prev_click)
    scaleChk.observe(changed)
    mesDrop.observe(meses_dropdown)
    estacionDrop.observe(estacion_dropdown)
    
    H = widgets.HBox([prevB,mesDrop,nextB, scaleChk], layout={'width':'80%'})
    V_in = widgets.VBox([output,H,estacionDrop], layout={'border': '1px solid black'})
    
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de caja de velocidad del viento mensual:
        A pesar de que ya se ha realizado una visualizacion con la velocidad del viento, esta permite visualizar el comportamiento general y las anomalias en la velocidad del viento con más detalle de una estacion determinada durante un mes y el mes previo/siguiente.
        Con esta visualizacion uno puede hacerse una idea de los periodos en los cuales se puede aprovechar de forma consistente un aerogenerador en un sector determinado.
        En caso de que hubieran demasiadas anomalias en la velocidad del viento, la generacion de electricidad de un aerogenerador seria demasiado inconsistente como para poder ser utilizado con efectividad.
        ''')
    V = widgets.VBox([V_in, textOut], layout={'width':'33%'})

    return V
    #display(V,textOut)

def scatter_vientoHum():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    colours = {}
    s_ids = ids
    s_opt = meses[0]
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()

    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[0],
        description='Periodo:',
        disabled=False,
        layout={'width':'90%'},
        style={'description_width': '30%'}
    )
    
    def meses_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal s_ids, s_opt
            s_opt = change['new']
            #est = estDf[estDf['momento'].dt.month == mes+1]
            replot(s_ids, s_opt)
    
    
    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            #display(sel_data)
            mesIndex = meses.index(option)
            month_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
            for id in selected_ids:
                filtered_df = month_df.query('ID_Estacion == '+id)
                plt.scatter(filtered_df["ff_Valor(m/s)"], filtered_df["HR_Valor(%)"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
            plt.xlim(0,50)
            plt.ylim(0,100)
            plt.xlabel("Velocidad del viento(m/s)")
            plt.ylabel("Humedad relativa del aire(%)")
            plt.suptitle("Relacion de niveles de Humedad y viento - Periodo "+option+"\nAño 2023")
            plt.legend()
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    mesDrop.observe(meses_dropdown)

    replot(s_ids, s_opt)
    
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([accordion, mesDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de humedad y velocidad del viento a nivel mensual:
        Permite ver una relacion entre el viento y los nieveles de humedad en el aire. Se pueden ver diferentes patrones provenientes de diferentes estaciones.
        El viento lleva consigo masas de aire que pueden contener niveles de humedad variable. Esto puede impactar tanto en paneles solares como en aerogeneradores de diferentes formas:
        - Condensacion del agua en la superficie de los paneles solares, disminuyendo la energia generada.
        - Altos niveles de humedad afectan en la densidad del aire, disminuyendo la energia generada por un aerogenerador.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(V)
    return V

#dim_lluvia()
#scatter_vientoHum()

def plotTimeSlider_viento():
    #Velocidad del viento promedio del año, medida por mes.
    global labels
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    start_date = min(bigDf['momento'])
    end_date = max(bigDf['momento'])
    s_d = start_date
    e_d = end_date
    selected = copy.deepcopy(ids)

    lineOutput = widgets.Output()

    dates = pd.date_range(start_date, end_date, freq='h')
    options = [(date.strftime('%d/%m/%Y'), date) for date in dates]
    index = (0, len(options)-1)
    
    selection_range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=index,
        description='Fechas',
        orientation='horizontal',
        layout={'width': '80%'}
    )

    def replot(labels, start, end):
        with lineOutput:
            #display(date_range)
            lineOutput.clear_output(wait=True)
            plt.figure(figsize=(10,4))
            filtered_df = bigDf.loc[(bigDf['momento'] >= start.strftime('%Y-%m-%d')) & (bigDf['momento'] < end.strftime('%Y-%m-%d'))]
            temp = pd.DataFrame(columns=ids)
            for id in labels:
                est = filtered_df.query('ID_Estacion == '+id)
                plt.plot(est['momento'], est['ff_Valor(m/s)'], color=colours[id], label = id,linewidth=0.75)
            plt.ylabel("Velocidad del viento (m/s)")
            plt.xticks(rotation=90)
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.suptitle("Linea de tiempo de velocidad del viento\nAño 2023")
            plt.grid()
            plt.show()
    
    def set_dateRange(date_range):
        nonlocal s_d, e_d
        s_d = date_range["new"][0]
        e_d = date_range["new"][1]
        replot(labels, s_d, e_d)
    
    def checkboxes(_):
        nonlocal s_d, e_d, selected
        selected = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(selected, s_d, e_d)
        
    replot(labels, s_d, e_d)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    
    selection_range_slider.observe(set_dateRange, 'value')
    
    V_in = widgets.VBox(chkbxs, layout={'width':'80%'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'20%'})
    H = widgets.HBox([lineOutput, ac_chk])
    V_in2 = widgets.VBox([H,selection_range_slider], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea de velocidad del viento en relacion al tiempo:
        Permite ver el comportamiento de la velocidad del viento a lo largo del tiempo de cada estacion. Es posible ajustar el rango de tiempo a visualizar del grafico.
        La escala se ajustara automaticamente en base a los datos visualizados. Esto se debe a que hay pocos datos cuyos valores estan muy fuera de lo normal, causando que no pueda distinguirse bien el comportamiento del resto de las lineas.
        La idea de este grafico es poder ver el comportamiento de la velocidad del viento a lo largo del año de todas las estaciones y comparar entre diferentes estaciones cual es el sector mas conveniente para instalar un aerogenerador.
        ''')
    V = widgets.VBox([V_in2, textOut], layout={'width':'33%'})

    return V
    #display(V)

In [4]:
def scatter_tempRadiacion():
    colours = {}
    s_ids = ids
    s_opt = "Ambos"
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        colours[id+"_line"] = (colours[id+"_scatter"][2],colours[id+"_scatter"][1],colours[id+"_scatter"][0])
    opcDrop = widgets.Dropdown(
        options=["Ambos","Solo Regresion Polinomial", "Solo Dispersion"],
        value='Ambos',
        description='Visualizar',
        disabled=False,
        layout={'width':'90%'}
    )
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()

    def opcDropChange(change):
        if(change['type'] == 'change' and change['name'] == 'value'):
            nonlocal s_ids, s_opt
            s_opt = change['new']
            replot(s_ids, s_opt)

    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,6))
            #display(sel_data)
            for id in selected_ids:
                filtered_df = bigDf.query('ID_Estacion == '+id)
                if(option == 'Solo Dispersion' or option == 'Ambos'):
                    plt.scatter(filtered_df["Ts_Valor(°C)"], filtered_df["RadGInst (watt/m2)"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
                if(option == 'Solo Regresion Polinomial' or option == 'Ambos'):
                    lrDf = pd.DataFrame()
                    lrDf["x"] = filtered_df["Ts_Valor(°C)"]
                    lrDf["y"] = filtered_df["RadGInst (watt/m2)"]
                    lrDf = lrDf.dropna(axis=0)
                    X_axis=pd.DataFrame(lrDf["x"])
                    Y_axis=pd.DataFrame(lrDf["y"])
                    poly = PolynomialFeatures(degree=3, include_bias=False)
                    poly_features = poly.fit_transform(X_axis.values.reshape(-1, 1))
                    lmDict[id] = LinearRegression()
                    models = lmDict[id].fit(poly_features, Y_axis)
                    plt.plot(X_axis,models.predict(poly_features),color=colours[id+"_line"], label = id+"_RP",linewidth=1.5)
            plt.xlim(-20,35)
            plt.ylim(0,1400)
            plt.xlabel("Temperatura(C°)")
            plt.ylabel("Radiacion Global Instantanea(watt/m2)")
            plt.suptitle("Relacion de niveles de Radiacion y temperatura\nAño 2023")
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    opcDrop.observe(opcDropChange)
    replot(s_ids, s_opt)
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([accordion] + [opcDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput,V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de temperatura y radiacion + Regresion Polinomial:
        Permite visualizar la relacion entre la temperatura y la radiacion medida por cada estacion. Incluye una recta generada por un modelo de regresion polinomial de los datos recolectados de cada estacion.
        La idea de este grafico era ver si existia alguna relacion entre radiacion y temperatura, motivo por el cual se utilizo un modelo de regresion polinomial para experimentar. Previamente se habia utilizado un modelo de regresion lineal.
        El modelo no es muy preciso.
        ''')

    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    return V #display(H,textOut)

    
def plotTimeSlider_rad():
    #Velocidad del viento promedio del año, medida por mes.
    global labels
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    start_date = min(bigDf['momento'])
    end_date = max(bigDf['momento'])
    s_d = start_date
    e_d = end_date
    selected = copy.deepcopy(ids)

    lineOutput = widgets.Output()

    dates = pd.date_range(start_date, end_date, freq='h')
    options = [(date.strftime('%d/%m/%Y'), date) for date in dates]
    index = (0, len(options)-1)
    
    selection_range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=index,
        description='Fechas',
        orientation='horizontal',
        layout={'width': '80%'}
    )

    def replot(labels, start, end):
        with lineOutput:
            #display(date_range)
            lineOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            filtered_df = bigDf.loc[(bigDf['momento'] >= start.strftime('%Y-%m-%d')) & (bigDf['momento'] < end.strftime('%Y-%m-%d'))]
            temp = pd.DataFrame(columns=ids)
            for id in labels:
                est = filtered_df.query('ID_Estacion == '+id).dropna()
                plt.plot(est['momento'], est['RadGInst (watt/m2)'], color=colours[id], label = id,linewidth=0.75)
            plt.ylabel("Radiacion global instantanea (watt/m2)")
            plt.xticks(rotation=90)
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.suptitle("Linea de tiempo de radiacion instantanea\nAño 2023")
            plt.grid()
            plt.show()
    
    def set_dateRange(date_range):
        nonlocal s_d, e_d, selected
        s_d = date_range["new"][0]
        e_d = date_range["new"][1]
        replot(selected, s_d, e_d)
    
    def checkboxes(_):
        nonlocal s_d, e_d, selected
        selected = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(selected, s_d, e_d)
        
    replot(labels, s_d, e_d)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    
    selection_range_slider.observe(set_dateRange, 'value')
    
    V_in = widgets.VBox(chkbxs, layout={'width':'80%'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'20%'})
    H = widgets.HBox([lineOutput,ac_chk])
    V_in2 = widgets.VBox([H,selection_range_slider], layout={'border': '1px solid black'})
    

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea de radiacion instantanea en relacion al tiempo:
        Nos permite ver los niveles de radiacion registrados a lo largo del año por cada estacion. Se pueden ver los periodos de niveles de radiacion maxima y minima en cada estacion son consistentes entre cada estacion. Tambien puede verse a grandes rasgos, cuales estaciones registran mayores niveles de radiacion entre si.''')

    V = widgets.VBox([V_in2,textOut], layout={'width':'33%'})
    #display(V,textOut)
    return V

def pie_radiacionTotal():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
    mesIndex = 0
    radiacionTotal = {}
    colours = {}
    for i in range(len(labels)):
        radiacionTotal[labels[i]] = (bigDf.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
        colours[labels[i]] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    selected = copy.deepcopy(labels)
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in radiacionTotal.keys()]
    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[len(meses) - 1],
        description='Periodo:',
        disabled=False,
        layout={'width':'90%'}
    )
    s_data = radiacionTotal.keys()
    s_vals = radiacionTotal.values()
    s_tag = meses[len(meses) - 1]
    scatterOutput = widgets.Output()
        
    def my_fmt(x):
        return '{:.4f}%\n({:.0f} watt/m2)'.format(x, sum(radiacionTotal.values())*x/100)
        
    def meses_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal s_data, s_vals, s_tag, radiacionTotal
            if(change['new'] == 'Anual'):
                for i in range(len(labels)):
                    radiacionTotal[labels[i]] = (bigDf.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
            else:
                mesIndex = meses.index(change['new'])
                f_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
                for i in range(len(labels)):
                    radiacionTotal[labels[i]] = (f_df.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
            s_data = [label for label,chk in zip(radiacionTotal.keys(),chkbxs) if chk.value]
            s_vals = [radiacionTotal[label] for label in s_data]
            s_tag = change['new']
            #est = estDf[estDf['momento'].dt.month == mes+1]
            replot(s_data, s_vals, s_tag)
    
    def replot(data, values, tag):
        nonlocal colours
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,6))
            plt.pie(values, colors=[colours[key] for key in data], labels=data,autopct=my_fmt, radius=1.75, pctdistance=1.15, labeldistance=1.3)
            plt.legend(title = "ID Estacion")
            ax = plt.gca()
            handles, labels = ax.get_legend_handles_labels()
            patch = mpatches.Patch(color='white', label="Total acumulado(visualizado):\n"+str(sum(values))+" watt/m2")
            handles.append(patch) 
            plt.legend(handles=handles, bbox_to_anchor=(2.04, 1.5), borderaxespad=0)
            plt.title("Radiacion Global Instantanea acumulada - Año 2023\nPeriodo "+tag, pad=175)
            plt.show()
    
    def checkboxes(_):
        nonlocal s_data, s_vals, s_tag
        s_data = [label for label,chk in zip(radiacionTotal.keys(),chkbxs) if chk.value]
        s_vals = [radiacionTotal[label] for label in s_data]
        replot(s_data,s_vals,s_tag)

    replot(radiacionTotal.keys(), radiacionTotal.values(), meses[len(meses) - 1])
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    mesDrop.observe(meses_dropdown)

    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([ac_chk, mesDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de torta de la radiacion acumulada durante un periodo:
        Permite visualizar la radiacion acumulada que ha sido registrada por todas las estaciones y el porcentaje que aporta cada una, durante un periodo de un mes determinado o durante el año.
        La idea de este grafico es poder determinar solo por la cantidad de radiacion que ha sido registrada, si es que es conveniente utilizar paneles solares en un determinado sector, segun la estacion que este asignada a este.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    
    return V    #display(H,textOut)


#dim_lluvia()

In [5]:
def bar_precipitacion():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
    precTotal = {}
    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[len(meses)-1],
        description='Periodo:',
        disabled=False,
        layout={'width':'30%'},
        style={'description_width': '35%'}
    )
    barOutput = widgets.Output()
    s_max = 0
    for id in ids:
        precTotal[id] = []
        for mes in meses:
            if(mes != 'Anual'):
                mesIndex = meses.index(mes)
                f_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
                temp = f_df.query('ID_Estacion == '+id).dropna()["RR6_Valor(mm)"].sum()
                if(temp > s_max):
                    s_max = temp
                precTotal[id].append(temp)
            else:
                precTotal[id].append(bigDf.query('ID_Estacion == '+id).dropna()["RR6_Valor(mm)"].sum())

    def mes_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal precTotal, s_max
            mesIndex = meses.index(change['new'])
            values = []
            for id in ids:
                values.append(precTotal[id][mesIndex])
            
            replot(precTotal.keys(), values, change['new'])
    
    def replot(labels,values,tag):
        nonlocal s_max
        with barOutput:
            barOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            plt.bar(labels, values)
            if(tag!='Anual'):
                plt.ylim(0,s_max)
            plt.ylabel("Precipitacion acumulada (mm)")
            plt.xlabel("Estaciones")
            plt.suptitle("Precipitacion acumulada por estacion - Periodo "+tag+"\nAño 2023")
            plt.grid()
            plt.show()    

    values = []
    for id in ids:
        values.append(precTotal[id][len(meses) - 1])
            
    replot(precTotal.keys(), values, meses[len(meses) - 1])
    #print(precTotal)

    mesDrop.observe(mes_dropdown)

    H = widgets.HBox([barOutput, mesDrop], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de barra de precipitacion acumulada por cada estacion a lo largo del año:
        Permite ver cuales son los niveles de precipitacion durante periodos de un mes determinado y eL total del año. Generalmente, precipitaciones indican niveles de nubosidad considerables, los cuales podrian afectar la cantidad de energia generada por un panel solar si se quiere instalar uno.
        ''')

    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(V,textOut)
    return V

def plot_avgHr():
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    meanDf = pd.DataFrame(columns=ids)
    err = True
    selected=ids
    
    for x in range(1,13):
        filtered_df = bigDf[bigDf['momento'].dt.month == x]
        temp = pd.DataFrame(columns=ids)
        for id in ids:
            est = filtered_df.query('ID_Estacion == '+id).dropna()
            #RadGInst (watt/m2)
            temp[id] = [est["HR_Valor(%)"].mean()]
            temp[id+"_std"] = [est["HR_Valor(%)"].std()]
        if(meanDf.size == 0):
            meanDf = pd.concat([temp], ignore_index=True)
        else:
            meanDf = pd.concat([meanDf,temp], ignore_index=True)

    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%','background':'red'}) for label in labels]
    errChk = widgets.Checkbox(description="Desviacion estandar?", value=err, indent=False, layout={'width':'90%'})
    plotOutput = widgets.Output()

    def errChange(change):
        nonlocal err, selected
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            err = not err
            replot(selected, err)
    
    def checkboxes(_):
        nonlocal err, selected
        selected = [label for label,chk in zip(ids,chkbxs) if chk.value]
        replot(selected, err)
            
    def replot(selected, dv):
        with plotOutput:
            plotOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            for id in selected:
                if(dv):
                    plt.errorbar(meses, meanDf[id], meanDf[id+"_std"], color=colours[id], capsize=5, fmt="-o", elinewidth=1, label = id)
                else:
                    plt.plot(meses, meanDf[id], label = id, color=colours[id], marker='o') 
            plt.xlabel("2023")
            plt.ylabel("Humedad relativa del aire(%)")
            plt.grid()
            plt.ylim(0,100)
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.suptitle("Humedad relativa promedio mensual\nAño 2023")
            plt.show()

    replot(selected,err)

    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    errChk.observe(errChange)
        
    V_in = widgets.VBox(chkbxs)
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'90%'})
    V_in2 =  widgets.VBox([ac_chk,errChk])
    H = widgets.HBox([plotOutput,V_in2], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea con desviacion estandar de la humedad relativa promedio mensual:
        Permite ver los niveles de humedad promedio que detecta cada estacion durante todos los meses. Tambien muestra que tan consistente es el valor promedio, visualizando una barra da error.
        Los niveles de humedad pueden indicar posible nubosidad, afectando la efectividad de un panel solar. Tambien puede afectar directamente en el panel solar en forma de agua acumulada en la superficie del panel por condensacion.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    return V


def scatter_tempHum():
    colours = {}
    s_ids = ids
    s_opt = "Ambos"
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        colours[id+"_line"] = (colours[id+"_scatter"][2],colours[id+"_scatter"][1],colours[id+"_scatter"][0])
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()


    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            #display(sel_data)
            for id in selected_ids:
                filtered_df = bigDf.query('ID_Estacion == '+id)
                plt.scatter(filtered_df["Ts_Valor(°C)"], filtered_df["HR_Valor(%)"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
            plt.xlim(-15,35)
            #plt.ylim(0,35)
            plt.xlabel("Temperatura(C°)")
            plt.ylabel("Humedad relativa del aier(%)")
            plt.suptitle("Relacion de niveles de Humedad relativa del aire y temperatura\nAño 2023")
            plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0)
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    replot(s_ids, s_opt)
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    #V_in2 = widgets.VBox([accordion], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, accordion], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de temperatura y humedad relativa del aire:
        Permite ver la relacion entre temperatura y la humedad del aire segun la estacion. Puede observarse una tendencia de disminucion de la humedad en cuanto la temperatura comienza a in
        Debido a que las estaciones estan situadas en diferentes sectores del pais, puede observarse a que niveles de temperatura la humedad comienza a disminuir en cada sector.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(H,textOut)
    return V
    

#dim_lluvia()


In [6]:
def plot_avgPotenciaSolar():
    colours = {}
    paneles = []
    htmlStyling = "<style>"


    for x in range(solar.shape[0]):
        paneles.append("Panel Solar "+str(x))
    for p in range(len(paneles)):
        colours[paneles[p]] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        a = pltcolors.rgb2hex(colours[paneles[p]], keep_alpha=False)
        htmlStyling = htmlStyling + """.solar-"""+str(p)+""" label {
        color: """+a+""" !important;
        font-weight: bold !important;
        }"""

    htmlStyling = htmlStyling + "</style>"

    display(widgets.HTML(htmlStyling))
    
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]

    err = True
    selected=paneles
    estacion=ids[0]
    meanDf = pd.DataFrame(index=range(solar.shape[0]), columns=meses)
    
    def recalc(queryID):
        nonlocal meses, meanDf
        meanDf = pd.DataFrame(index=range(solar.shape[0]), columns=meses)
        est = bigDf.query('ID_Estacion == '+queryID)
        for x in range(1,13):
            filtered_df = est[est['momento'].dt.month == x]
            for y in range(len(paneles)):
                meanDf.loc[y,meses[x-1]] = filtered_df[paneles[y]].mean()
        
        meanDf = meanDf.T
        
    estacionDrop = widgets.Dropdown(
        options=ids,
        value=ids[0],
        description='Estacion:',
        disabled=False,
    )
    recalc(estacion)
    chkbxs = [widgets.Checkbox(description=paneles[p], value=True, indent=False, layout={'width':'90%'}, _dom_classes=["solar-"+str(p)]) for p in range(len(paneles))]
    errChk = widgets.Checkbox(description="Desviacion estandar?", value=err, indent=False, layout={'width':'90%'})
    plotOutput = widgets.Output()

    def estacion_dropdown(change):
        nonlocal estacion, selected
        if(change['type'] == 'change' and change['name'] == 'value'):
            estacion = change['new']
            recalc(estacion)
            replot(selected)
            print("changed to "+ change['new'] +" which is on index "+str(ids.index(change['new'])))
    
    def errChange(change):
        nonlocal err, selected
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            err = not err
            replot(selected)
    
    def checkboxes(_):
        nonlocal err, selected
        selected = [label for label,chk in zip(paneles,chkbxs) if chk.value]
        replot(selected)
            
    def replot(selected):
        with plotOutput:
            plotOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            for i in selected:
                    tmp = i.split(" ")
                    plt.plot(meses, meanDf[int(tmp[len(tmp) - 1])], label = i, color=colours[i], marker='o') 
            plt.xlabel("2023")
            plt.ylabel("Potencia generada(W)")
            plt.grid()
            plt.ylim(0,250)
            plt.suptitle("Potencia solar promedio mensual de paneles solares Año 2023\n"+estaciones[estacion][0] +"- ID:"+estacion)
            plt.show()

    replot(selected)

    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    errChk.observe(errChange)
    estacionDrop.observe(estacion_dropdown)

    V_in = widgets.VBox(chkbxs)
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Paneles solares',), layout={'width':'100%'})
    V_in2 =  widgets.VBox([ac_chk,estacionDrop])
    H = widgets.HBox([plotOutput,V_in2], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea con desviacion estandar de la velocidad del viento promedio mensual''')
    V = widgets.VBox([H,textOut], layout={'width':'100%'})
    return V


In [7]:
def plot_avgPotenciaEolica():
    colours = {}
    paneles = []
    htmlStyling = "<style>"
    
    """
    <style>
    .my-label-style label {
        color:  !important;
        font-weight: bold !important;
    }
    </style>
    """
    
    for x in range(aero.shape[0]):
        paneles.append("Aerogenerador "+str(x))
    for p in range(len(paneles)):
        colours[paneles[p]] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        a = pltcolors.rgb2hex(colours[paneles[p]], keep_alpha=False)
        htmlStyling = htmlStyling + """.aero-"""+str(p)+""" label {
        color: """+a+""" !important;
        font-weight: bold !important;
        }"""


    htmlStyling = htmlStyling + "</style>"

    display(widgets.HTML(htmlStyling))
    
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    err = True
    selected=paneles
    estacion=ids[0]
    meanDf = pd.DataFrame(index=range(solar.shape[0]), columns=meses)
    
    def recalc(queryID):
        nonlocal meses, meanDf
        meanDf = pd.DataFrame(index=range(solar.shape[0]), columns=meses)
        est = bigDf.query('ID_Estacion == '+queryID)
        for x in range(1,13):
            filtered_df = est[est['momento'].dt.month == x]
            for y in range(len(paneles)):
                meanDf.loc[y,meses[x-1]] = filtered_df[paneles[y]].mean()
        
        meanDf = meanDf.T
        
    estacionDrop = widgets.Dropdown(
        options=ids,
        value=ids[0],
        description='Estacion:',
        disabled=False,
    )
    recalc(estacion)
    chkbxs = [widgets.Checkbox(description=paneles[p], value=True, indent=False, layout={'width':'90%'}, _dom_classes=["solar-"+str(p)]) for p in range(len(paneles))]
    errChk = widgets.Checkbox(description="Desviacion estandar?", value=err, indent=False, layout={'width':'90%', 'background':'red'})
    plotOutput = widgets.Output()

    def estacion_dropdown(change):
        nonlocal estacion, selected
        if(change['type'] == 'change' and change['name'] == 'value'):
            estacion = change['new']
            recalc(estacion)
            replot(selected)
            print("changed to "+ change['new'] +" which is on index "+str(ids.index(change['new'])))
    
    def errChange(change):
        nonlocal err, selected
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            err = not err
            replot(selected)
    
    def checkboxes(_):
        nonlocal err, selected
        selected = [label for label,chk in zip(paneles,chkbxs) if chk.value]
        replot(selected)
            
    def replot(selected):
        with plotOutput:
            plotOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            for i in selected:
                    tmp = i.split(" ")
                    plt.plot(meses, meanDf[int(tmp[len(tmp) - 1])], label = i, color=colours[i], marker='o') 
            plt.xlabel("2023")
            plt.ylabel("Potencia generada(W)")
            plt.grid()
            plt.ylim(0,1900)
            plt.suptitle("Potencia eolica de aerogeneradroes promedio mensual Año 2023\n"+estaciones[estacion][0]+" - ID:"+estacion)
            plt.show()

    replot(selected)

    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    errChk.observe(errChange)
    estacionDrop.observe(estacion_dropdown)

    V_in = widgets.VBox(chkbxs)
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Aerogeneradores',), layout={'width':'100%'})
    V_in2 =  widgets.VBox([ac_chk,estacionDrop])
    H = widgets.HBox([plotOutput,V_in2], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea con desviacion estandar de la velocidad del viento promedio mensual''')
    V = widgets.VBox([H,textOut], layout={'width':'100%'})
    return V


In [8]:
def dim_lluvia():
    H = widgets.HBox([plot_avgHr(), scatter_tempHum(), bar_precipitacion()], layout={'height':'600px','width':'3000px'})

    display(H)
    
def dim_viento():
    H = widgets.HBox([plot_avgViento(),boxPlot_viento(), scatter_vientoHum(), plotTimeSlider_viento()], layout={'height':'600px','width':'4200px'})

    display(H)

def dim_radiacion():
    H = widgets.HBox([pie_radiacionTotal(), scatter_tempRadiacion(), plotTimeSlider_rad()], layout={'height':'800px','width':'3000px'})

    display(H)

def dim_potencia():
    H = widgets.HBox([plot_avgPotenciaSolar(), plot_avgPotenciaEolica()], layout={'height':'700px','width':'3000px'})

    display(H)

In [9]:
tabs = ["Viento", "Radiacion", "Agua", "Potencia"]

sub_tab=[widgets.Output() for i in range(len(tabs))]
tab = widgets.Tab(sub_tab)
for i in range (len(tabs)):
    tab.set_title(i,tabs[i])
    with sub_tab[i]:
        if(i == 0):
            dim_viento()
        elif(i == 1):
            dim_radiacion()
        elif(i == 2):
            dim_lluvia()
        else:
            dim_potencia()
display(tab)

In [10]:
dim_potencia()

HTML(value='<style>.solar-0 label {\n        color: #9e51d5 !important;\n        font-weight: bold !important;…

HTML(value='<style>.aero-0 label {\n        color: #81d832 !important;\n        font-weight: bold !important;\…

In [11]:
dim_lluvia()

In [12]:
dim_radiacion()

In [13]:
dim_viento()

In [14]:
bigDf["ff_Valor(m/s)"].max()

46.814404